# Recurrent neural network: IMDB example

In [24]:
import numpy as np
import pandas as pd
import matplotlib as plt

from keras.utils import plot_model
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import SimpleRNN, Embedding, Input
from keras.layers.recurrent import LSTM, GRU
from keras.models import Model

In [2]:
# For reproducibility:
np.random.seed(1234)

In [6]:
import os

img_path = './img'

if not os.path.isdir(img_path):
    os.mkdir(img_path)

## Global parameters and helper functions

In [7]:
size_vocabulary = 10000
max_sentence_len = 500 
embedding_dim = 128

batch_size=128

In [8]:
# TODO: Change epochs to 4
def train_and_eval(model, input_train, y_train, input_test, y_test):
    model.fit(input_train,  
              y_train,
              epochs=1,
              batch_size=batch_size,
              validation_split=0.2)
    
    score, acc = model.evaluate(input_test, y_test)
    
    print('Test score:', score)
    print('Test accuracy:', acc)

In [9]:
def display_model(model):
    from IPython.display import Image
    from IPython.core.display import HTML 
    
    path = './img/{}.png'.format(model.name)
    plot_model(model, to_file=path)
    
    return Image(url=path)

## The data

In [10]:
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_sentence_len)

In [11]:
print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')

print('Shape train sequences:', input_train.shape)
print('Shape test sequences:', input_train.shape)
print('Shape train labels:', y_train.shape)
print('Shape test labels:', y_test.shape)

25000 train sequences
25000 test sequences
Shape train sequences: (25000,)
Shape test sequences: (25000,)
Shape train labels: (25000,)
Shape test labels: (25000,)


In [12]:
pd.DataFrame(input_train, columns=["input_train"]).head()

,input_train
0,"[1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 458, 2, 66..."
1,"[1, 194, 2, 194, 2, 78, 228, 5, 6, 2, 2, 2, 13..."
2,"[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 2..."
3,"[1, 4, 2, 2, 33, 2, 4, 2, 432, 111, 153, 103, ..."
4,"[1, 249, 2, 7, 61, 113, 10, 10, 13, 2, 14, 20,..."


## Preprocessing: Padding

In [13]:
input_train = sequence.pad_sequences(input_train, maxlen=max_sentence_len)
input_test = sequence.pad_sequences(input_test, maxlen=max_sentence_len)

In [14]:
print('Pad sequences (shape: samples x time)')
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

Pad sequences (shape: samples x time)
input_train shape: (25000, 500)
input_test shape: (25000, 500)


In [15]:
input_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

If batch of shape `(n_samples, max_sentence_length)` is fed to 
this layer, the output will have shape `(n_samples, max_sentence_length, embedding_dim)`:

## Simple RNN using the Sequential API

In [16]:
model = Sequential(name="simple_rnn_sequential")
model.add(Embedding(size_vocabulary, embedding_dim, name="embedding_layer")) 
model.add(SimpleRNN(units=embedding_dim, name="simple_rnn_layer"))
model.add(Dense(1, activation='sigmoid', name="dense_layer"))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, None, 128)         1280000   
_________________________________________________________________
simple_rnn_layer (SimpleRNN) (None, 128)               32896     
_________________________________________________________________
dense_layer (Dense)          (None, 1)                 129       
Total params: 1,313,025
Trainable params: 1,313,025
Non-trainable params: 0
_________________________________________________________________


In [17]:
display_model(model)

In [14]:
train_and_eval(model, input_train, y_train, input_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/1
25000/25000 [==============================] - 36s 1ms/step
Test score: 0.6672063104057312
Test accuracy: 0.58304


## Simple RNN using the Functional API

In [27]:
inputs = Input(shape=(max_sentence_len,), name="input_layer")
embedding_layer = Embedding(size_vocabulary, embedding_dim, name="embedding_layer")(inputs)
simple_rnn_layer = SimpleRNN(units=embedding_dim, name="simple_rnn_layer")(embedding_layer)
predictions = Dense(1, activation='sigmoid', name="dense_layer")(simple_rnn_layer)

model = Model(inputs=inputs, 
              outputs=predictions,
              name="simple_rnn_functional")

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 500)               0         
_________________________________________________________________
embedding_layer (Embedding)  (None, 500, 128)          1280000   
_________________________________________________________________
simple_rnn_layer (SimpleRNN) (None, 128)               32896     
_________________________________________________________________
dense_layer (Dense)          (None, 1)                 129       
Total params: 1,313,025
Trainable params: 1,313,025
Non-trainable params: 0
_________________________________________________________________


In [28]:
display_model(model)

In [16]:
train_and_eval(model, input_train, y_train, input_test, y_test)

Train on 20000 samples, validate on 5000 samples
Epoch 1/1
25000/25000 [==============================] - 38s 2ms/step
Test score: 0.6479414934921265
Test accuracy: 0.62892


## LSTM using the Sequential API

In [29]:
model = Sequential(name="lstm_sequential")
model.add(Embedding(size_vocabulary, embedding_dim, name="embedding_layer")) 
model.add(LSTM(units=embedding_dim, name="lstm_layer"))
model.add(Dense(1, activation='sigmoid', name="dense_layer"))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, None, 128)         1280000   
_________________________________________________________________
lstm_layer (LSTM)            (None, 128)               131584    
_________________________________________________________________
dense_layer (Dense)          (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [30]:
display_model(model)

In [ ]:
train_and_eval(model, input_train, y_train, input_test, y_test)

# GRU using the Sequential API

In [31]:
model = Sequential(name="gru_sequential")
model.add(Embedding(size_vocabulary, embedding_dim, name="embedding_layer")) 
model.add(GRU(units=embedding_dim, name="lstm_layer"))
model.add(Dense(1, activation='sigmoid', name="dense_layer"))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, None, 128)         1280000   
_________________________________________________________________
lstm_layer (GRU)             (None, 128)               98688     
_________________________________________________________________
dense_layer (Dense)          (None, 1)                 129       
Total params: 1,378,817
Trainable params: 1,378,817
Non-trainable params: 0
_________________________________________________________________


In [32]:
display_model(model)